# Imports

In [1]:
import numpy as np
import os
import cv2

# Generate data

In [2]:
#A useful function for testing rgb
def make_rgb(random= False):
    """
    Generates an rgb tuple
    """
    if random:
        values = np.floor(np.random.random(3)*255)
    else:
        values = [255,255,255]
    rgb = np.array(values).reshape(3,)
    rgb = np.uint8(rgb)
    return rgb

make_rgb(1)

array([190, 224, 151], dtype=uint8)

In [3]:
rgb = make_rgb()
print(rgb)

[255 255 255]


# An example function

In [4]:
rgb = make_rgb()
print(rgb)#This is the first attempt I made to convert the data. 
def reducebits(rgb):
    rgb = np.array(rgb).copy() #do not alter the source
    rgb = np.uint8(rgb)
    rgb = rgb.flatten()
    if len(rgb)!=3:
        raise Exception("RGB is not an iterable with 3 channels")
        
    for i, channel in enumerate(rgb):
        if i == 2:
            #this is the blue channel
            rgb[i] = rgb[i]//16
        else:
            #these are the red and green channels
            rgb[i] = rgb[i]//32    
    return rgb        
        

[255 255 255]


In [5]:
a = reducebits(rgb)
print(a)

[ 7  7 15]


We still need to convert to bits and concatenate. But it turns out that numpy has a convenient function so we can use that instead

# Conversion

In [6]:
def convert_pixel_10bit(rgb, verbose= False):
    rgb = np.array(rgb).copy() #do not alter the source
    rgb = np.uint8(rgb)
    if len(rgb)!=3:
        raise Exception("RGB is not an iterable with 3 channels")
    pad = np.zeros(6, dtype=np.uint8)
        
    rgb = rgb.reshape(3,1)
    if verbose:
        print("Input")
        print(rgb)
        print()
    rgb = np.unpackbits(rgb, axis=1)
    rgb = np.concatenate([rgb[0][:3],rgb[1][:3],rgb[2][:4]])
    if verbose:
        print("10-bit")
        print(rgb)
        print()
    rgb = np.concatenate([pad, rgb])
    mask = np.array([2**i*x for i,x in enumerate(rgb[::-1])])
    rgb = np.sum(mask)
    if verbose:
        print("integer in 0-1023 range")
        print(rgb)
        print()
    return rgb 

In [8]:
rgb = make_rgb(random=True)
convert_pixel_10bit(rgb, verbose=True)

Input
[[ 95]
 [  0]
 [211]]

10-bit
[0 1 0 0 0 0 1 1 0 1]

integer in 0-1023 range
269



269